In [8]:
import requests
import defs
import pandas as pd

In [9]:
session = requests.Session()

In [10]:
ins_df = pd.read_pickle("instruments.pkl")

In [16]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,EUR_ZAR,CURRENCY,EUR/ZAR,-4,0.07
1,EUR_CHF,CURRENCY,EUR/CHF,-4,0.04
2,USD_PLN,CURRENCY,USD/PLN,-4,0.05
3,GBP_AUD,CURRENCY,GBP/AUD,-4,0.05
4,EUR_AUD,CURRENCY,EUR/AUD,-4,0.03
...,...,...,...,...,...
63,USD_CAD,CURRENCY,USD/CAD,-4,0.02
64,AUD_USD,CURRENCY,AUD/USD,-4,0.03
65,HKD_JPY,CURRENCY,HKD/JPY,-4,0.1
66,EUR_SEK,CURRENCY,EUR/SEK,-4,0.03


In [17]:
ins_df.name.unique()

array(['EUR_ZAR', 'EUR_CHF', 'USD_PLN', 'GBP_AUD', 'EUR_AUD', 'EUR_SGD',
       'NZD_CAD', 'TRY_JPY', 'CAD_HKD', 'USD_CHF', 'AUD_HKD', 'NZD_CHF',
       'AUD_CHF', 'GBP_CHF', 'USD_THB', 'GBP_CAD', 'EUR_HKD', 'CHF_JPY',
       'GBP_HKD', 'EUR_NZD', 'AUD_SGD', 'EUR_JPY', 'EUR_TRY', 'USD_JPY',
       'SGD_JPY', 'GBP_ZAR', 'ZAR_JPY', 'AUD_JPY', 'USD_ZAR', 'CAD_JPY',
       'NZD_HKD', 'USD_CZK', 'USD_DKK', 'USD_SEK', 'GBP_SGD', 'EUR_DKK',
       'CHF_ZAR', 'CAD_CHF', 'GBP_USD', 'USD_MXN', 'USD_HUF', 'USD_HKD',
       'EUR_USD', 'EUR_CAD', 'AUD_CAD', 'GBP_PLN', 'EUR_PLN', 'GBP_NZD',
       'EUR_HUF', 'EUR_NOK', 'CHF_HKD', 'EUR_GBP', 'AUD_NZD', 'CAD_SGD',
       'EUR_CZK', 'NZD_JPY', 'USD_TRY', 'GBP_JPY', 'SGD_CHF', 'USD_CNH',
       'USD_NOK', 'NZD_SGD', 'NZD_USD', 'USD_CAD', 'AUD_USD', 'HKD_JPY',
       'EUR_SEK', 'USD_SGD'], dtype=object)

In [18]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [19]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f'{p1}_{p2}'
        if pair in ins_df.name.unique():
            print(pair)
            
        

EUR_USD
EUR_GBP
EUR_JPY
EUR_CHF
EUR_NZD
EUR_CAD
USD_JPY
USD_CHF
USD_CAD
GBP_USD
GBP_JPY
GBP_CHF
GBP_NZD
GBP_CAD
CHF_JPY
NZD_USD
NZD_JPY
NZD_CHF
NZD_CAD
CAD_JPY
CAD_CHF


In [20]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [21]:
status_code, data = fetch_candles('EUR_USD', 10, 'H1')

In [25]:
def get_candles_df(json_response):

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [28]:
df_candles = get_candles_df(data)

In [29]:
df_candles

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-08-10T08:00:00.000000000Z,4969,1.10085,1.10266,1.10082,1.10186,1.10077,1.10258,1.10075,1.10179,1.10093,1.10273,1.10090,1.10193
1,2023-08-10T09:00:00.000000000Z,3192,1.10185,1.10248,1.10162,1.10208,1.10178,1.10240,1.10154,1.10199,1.10192,1.10257,1.10169,1.10216
2,2023-08-10T10:00:00.000000000Z,2843,1.10208,1.10280,1.10192,1.10278,1.10200,1.10273,1.10185,1.10271,1.10216,1.10288,1.10199,1.10285
3,2023-08-10T11:00:00.000000000Z,3484,1.10280,1.10298,1.10146,1.10167,1.10272,1.10291,1.10138,1.10159,1.10287,1.10305,1.10153,1.10175
4,2023-08-10T12:00:00.000000000Z,13087,1.10169,1.10652,1.10107,1.10418,1.10162,1.10643,1.10099,1.10410,1.10176,1.10662,1.10115,1.10427
5,2023-08-10T13:00:00.000000000Z,11724,1.10415,1.10458,1.10078,1.10396,1.10407,1.10451,1.10070,1.10389,1.10423,1.10465,1.10085,1.10404
6,2023-08-10T14:00:00.000000000Z,10067,1.10397,1.10440,1.10124,1.10187,1.10390,1.10432,1.10117,1.10180,1.10404,1.10448,1.10131,1.10194
7,2023-08-10T15:00:00.000000000Z,6392,1.10190,1.10222,1.10086,1.10153,1.10183,1.10214,1.10078,1.10145,1.10197,1.10230,1.10094,1.10161
8,2023-08-10T16:00:00.000000000Z,4055,1.10152,1.10194,1.10102,1.10134,1.10145,1.10185,1.10094,1.10126,1.10160,1.10202,1.10109,1.10142


In [30]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"his_data/{pair}_{granularity}.pkl")

In [31]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [32]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")

EUR_USD loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2022-12-20T01:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
USD_JPY loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
USD_CHF loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
USD_CAD loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
GBP_USD loaded 3999 candles from 2022-12-20T02:00:00.000000000Z to 2023-08-10T16:00:00.000000000Z
GBP_JPY loaded 3999 

In [33]:
pd.read_pickle(f'his_data/EUR_USD_H1.pkl')

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-12-20T02:00:00.000000000Z,3439,1.06052,1.06156,1.06032,1.06147,1.06043,1.06149,1.06024,1.06139,1.06062,1.06164,1.06040,1.06155
1,2022-12-20T03:00:00.000000000Z,18405,1.06148,1.06524,1.06022,1.06084,1.06139,1.06515,1.06014,1.06075,1.06156,1.06534,1.06031,1.06092
2,2022-12-20T04:00:00.000000000Z,8431,1.06082,1.06160,1.06038,1.06076,1.06074,1.06152,1.06029,1.06067,1.06090,1.06168,1.06046,1.06084
3,2022-12-20T05:00:00.000000000Z,6637,1.06074,1.06125,1.05790,1.05866,1.06067,1.06115,1.05783,1.05857,1.06080,1.06135,1.05798,1.05875
4,2022-12-20T06:00:00.000000000Z,8749,1.05868,1.06019,1.05831,1.05936,1.05858,1.06011,1.05823,1.05928,1.05877,1.06028,1.05839,1.05944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2023-08-10T12:00:00.000000000Z,13087,1.10169,1.10652,1.10107,1.10418,1.10162,1.10643,1.10099,1.10410,1.10176,1.10662,1.10115,1.10427
3995,2023-08-10T13:00:00.000000000Z,11724,1.10415,1.10458,1.10078,1.10396,1.10407,1.10451,1.10070,1.10389,1.10423,1.10465,1.10085,1.10404
3996,2023-08-10T14:00:00.000000000Z,10067,1.10397,1.10440,1.10124,1.10187,1.10390,1.10432,1.10117,1.10180,1.10404,1.10448,1.10131,1.10194
3997,2023-08-10T15:00:00.000000000Z,6392,1.10190,1.10222,1.10086,1.10153,1.10183,1.10214,1.10078,1.10145,1.10197,1.10230,1.10094,1.10161
